In [5]:
import requests
import os

# Function to download audio recordings of avian species from Xenocanto
def download_recordings(species_name, num_recordings=5, output_dir='audio'):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # API endpoint for searching recordings by species name
    api_url = f'https://www.xeno-canto.org/api/2/recordings?query={species_name}&num={num_recordings}'
    
    try:
        # Send HTTP GET request to the API
        response = requests.get(api_url)
        data = response.json()
        
        # Initialize counter for downloaded recordings
        count = 0
        
        # Loop through recordings and download audio files
        for idx, recording in enumerate(data['recordings']):
            # Check if the number of recordings downloaded for the species exceeds the limit
            if count >= num_recordings:
                break
            
            file_url = recording['file']
            file_name = f"{species_name}_{idx + 1}.mp3"
            file_path = os.path.join(output_dir, file_name)
            
            # Download audio file
            with open(file_path, 'wb') as file:
                audio_response = requests.get(file_url)
                file.write(audio_response.content)
            
            print(f"Downloaded: {file_name}")
            
            # Increment counter for downloaded recordings
            count += 1
    
    except Exception as e:
        print(f"Error: {e}")

# Example usage: Download 5 recordings of the "European Robin"
download_recordings('European Robin', num_recordings=5)


KeyboardInterrupt: 

In [24]:
import os
import librosa
import numpy as np

# Function to compute Mel spectrogram from audio clip
def compute_mel_spectrogram(audio_clip, sample_rate=22050, n_fft=2048, hop_length=512, n_mels=128):
    # Compute the short-time Fourier transform (STFT)
    stft = librosa.stft(audio_clip, n_fft=n_fft, hop_length=hop_length)
    
    # Convert the STFT magnitude to decibels (dB)
    stft_db = librosa.amplitude_to_db(np.abs(stft))
    
    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(S=stft_db, sr=sample_rate, n_mels=n_mels)
    
    return mel_spectrogram

# Function to process audio files in a directory
def process_audio_directory(input_dir, output_dir, sample_rate=22050, n_fft=2048, hop_length=512, n_mels=128):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(root, file)
                
                # Load audio clip
                audio_clip, _ = librosa.load(file_path, sr=sample_rate)
                
                # Compute Mel spectrogram
                mel_spectrogram = compute_mel_spectrogram(audio_clip, sample_rate=sample_rate, n_fft=n_fft,
                                                           hop_length=hop_length, n_mels=n_mels)
                
                # Get label from directory name
                label = os.path.basename(os.path.dirname(file_path))
                
                # Create output directory for label if it doesn't exist
                output_label_dir = os.path.join(output_dir, label)
                os.makedirs(output_label_dir, exist_ok=True)
                
                # Save Mel spectrogram
                output_file_path = os.path.join(output_label_dir, f"{os.path.splitext(file)[0]}.npy")
                np.save(output_file_path, mel_spectrogram)
                print(f"Saved Mel spectrogram: {output_file_path}")

# Example usage:
input_dir = '/home/mrrahul/code/project/bird-density-prediction/audio/'  # Directory containing audio files
output_dir = '/home/mrrahul/code/project/bird-density-prediction/mel'  # Directory to save Mel spectrograms

process_audio_directory(input_dir, output_dir)


/tmp/ipykernel_223083/690523048.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_clip, _ = librosa.load(file_path, sr=sample_rate)


EOFError: 